In [76]:
import pandas as pd
from glob import glob
from os.path import basename,splitext
from statistics import mean
pd.set_option('display.colheader_justify', 'center')

In [77]:
file2name = {
    'amazon0302':'Amazon',
    "com-youtube": 'Youtube',
    "soc-LiveJournal1": 'LiveJournal',
    "soc-pokec-relationships": 'Pokec',
    "com-orkut": 'Orkut',
    "ca-HepPh": 'NetPhy',
    "ca-HepTh": 'NetHEP',
    "com-dblp": 'DBLP',
    "ego-twitter": 'Twitter',
    "soc-Epinions1": 'Epinions',
    "soc-Slashdot0811": 'Slashdot0811',
    "soc-Slashdot0902": 'Slashdot0902',
    "soc-pokec-relationships":'Pokec',
    "com-friendster":'Friendster',
}

In [78]:
files = glob("../../fillresults/*.txt")
rows = []
d = {'0':'Naive', '1':'Sorted'}
for f in files:
    dataset, p, method, K, eps, R, s = splitext(basename(f))[0].split('_')
    dataset = dataset.split('.')[0]
    # print(dataset, p, method, K, eps, R, s)
    try:
        _, rate = open(f).readlines()[-1].split(':')

        row = [dataset, p, int(R), d[s], float(rate)]
        rows.append(row)
    except:
        pass
    # print(row)


In [79]:
df = pd.DataFrame(rows,columns=['Dataset','Setting','R','Approach','Fillrate'])
df = df[df.R==256].drop('R', axis=1)
df = df[df.Setting.isin(['0.005','0.01','0.1'])]
df = df.sort_values(by=['Dataset'])
df.replace({"Dataset": file2name},inplace=True)
df

,Dataset,Setting,Approach,Fillrate
0,Amazon,0.01,Naive,0.04
120,Amazon,0.005,Naive,0.04
3,Amazon,0.1,Sorted,0.50
121,Amazon,0.005,Sorted,0.07
2,Amazon,0.1,Naive,0.13
...,...,...,...,...
91,Pokec,0.1,Sorted,0.43
88,Pokec,0.01,Naive,0.04
89,Pokec,0.01,Sorted,0.08
142,Pokec,0.005,Naive,0.03


In [92]:
pivot = df.pivot_table(index=['Dataset'], columns=['Setting','Approach'],margins=True,
               margins_name='Mean',  # defaults to 'All'
               aggfunc=mean).iloc[: , :-1]
# pivot.style.format('{:,.2f}')
# pivot = 
# pivot.style.set_properties(
#     subset=['index'], 
#     **{'font-weight': 'bold'}
# )
latex_str = pivot.to_latex(float_format=lambda x: '{:,.0%}'.format(x),na_rep='-',multirow=True)
splits = latex_str.replace('{l}','{c}').split('\n')
a = splits[7:-3]
b = []
for i in a:
    cells = i.split('&')
    if cells[0]!= 'Mean':
        cells[0] = '\\tt{'+cells[0]+'}'; 
    cells = "&".join(cells)
    b.append(cells)
splits[7:-3] = b
print('\n'.join(splits))



\begin{tabular}{lrrrrrr}
\toprule
{} & \multicolumn{6}{c}{Fillrate} \\
Setting & \multicolumn{2}{c}{0.005} & \multicolumn{2}{c}{0.01} & \multicolumn{2}{c}{0.1} \\
Approach &  Naive   & Sorted & Naive & Sorted & Naive & Sorted \\
Dataset      &          &        &       &        &       &        \\
\midrule
\tt{Amazon       }&    4\%    &   7\%   &   4\%  &  11\%   &  13\%  &  50\%   \\
\tt{DBLP         }&    3\%    &   6\%   &   4\%  &   8\%   &  10\%  &  43\%   \\
\tt{Epinions     }&    3\%    &   6\%   &   4\%  &   8\%   &  10\%  &  43\%   \\
\tt{LiveJournal  }&    3\%    &   6\%   &   4\%  &   8\%   &  10\%  &  44\%   \\
\tt{NetHEP       }&    4\%    &   8\%   &   4\%  &  14\%   &  19\%  &  58\%   \\
\tt{NetPhy       }&    4\%    &   8\%   &   4\%  &  14\%   &  19\%  &  58\%   \\
\tt{Orkut        }&    3\%    &   6\%   &   4\%  &   8\%   &  10\%  &  44\%   \\
\tt{Pokec        }&    3\%    &   6\%   &   4\%  &   8\%   &  10\%  &  43\%   \\
\tt{Slashdot0811 }&    3\%    &   6\%   &   

In [81]:
pivot

Fillrate                                    
Setting        0.005           0.01         0.1           
Approach       Naive   Sorted Naive Sorted Naive   Sorted 
Dataset                                                   
Amazon        0.040000  0.070  0.04  0.110  0.13  0.500000
DBLP          0.030000  0.060  0.04  0.080  0.10  0.430000
Epinions      0.030000  0.060  0.04  0.080  0.10  0.430000
LiveJournal   0.030000  0.060  0.04  0.080  0.10  0.440000
NetHEP        0.040000  0.080  0.04  0.140  0.19  0.580000
NetPhy        0.040000  0.080  0.04  0.140  0.19  0.580000
Orkut         0.030000  0.060  0.04  0.080  0.10  0.440000
Pokec         0.030000  0.060  0.04  0.080  0.10  0.430000
Slashdot0811  0.030000  0.060  0.04  0.080  0.10  0.430000
Slashdot0902  0.030000  0.060  0.04  0.080  0.10  0.440000
Twitter       0.040000  0.070  0.04  0.110  0.13  0.480000
Youtube       0.030000  0.060  0.04  0.080  0.10  0.440000
Average       0.033333  0.065  0.04  0.095  0.12  0.468333

In [86]:
df2 = pd.read_csv('../../report.txt',delimiter=' ',names=['tag','Split','Naive','Sorted'],header=None)
df2['Dataset']=df2.tag.apply(lambda x: x.split('_')[0].split('.')[0])
df2['Setting']=df2.tag.apply(lambda x: x.split('_')[1])
df2

,tag,Split,Naive,Sorted,Dataset,Setting
0,amazon0302_0.01,1,0.95,0.96,amazon0302,0.01
1,amazon0302_0.01,2,0.81,0.51,amazon0302,0.01
2,amazon0302_0.01,4,0.61,0.28,amazon0302,0.01
3,amazon0302_0.01,8,0.37,0.16,amazon0302,0.01
4,amazon0302_0.1,1,1.00,1.00,amazon0302,0.1
...,...,...,...,...,...,...
299,soc-Slashdot0811_0.005,8,0.16,0.11,soc-Slashdot0811,0.005
300,soc-Slashdot0902_0.005,1,0.72,0.73,soc-Slashdot0902,0.005
301,soc-Slashdot0902_0.005,2,0.48,0.39,soc-Slashdot0902,0.005
302,soc-Slashdot0902_0.005,4,0.28,0.21,soc-Slashdot0902,0.005


In [89]:
df2 = df2[df2.Split!=1]
df2 = df2[df2.Setting.isin(['0.005','0.01','0.1'])]
df2.replace({"Dataset": file2name},inplace=True)
df2 = df2.sort_values(by=['Dataset'])
pivot2 = df2.drop(['tag'],axis=1).pivot_table(index=['Dataset'], columns=['Setting','Split'],margins=True,
               margins_name='Mean',  # defaults to 'All'
               aggfunc=mean)
pivot2 = pivot2.reorder_levels([1,2,0], axis=1).clip(0,1)
pivot2= pivot2.iloc[: , :-1]
pivot2.sort_index(axis=1, inplace=True)
pivot2


Setting        0.005                                                      \
Split            2                   4                   8                 
               Naive     Sorted    Naive     Sorted    Naive     Sorted    
Dataset                                                                    
Amazon        0.580000  0.440000  0.360000  0.230000  0.210000  0.120000   
DBLP          0.480000  0.390000  0.280000  0.210000  0.160000  0.110000   
Epinions      0.480000  0.390000  0.280000  0.210000  0.160000  0.110000   
Friendster    0.480000  0.390000  0.280000  0.210000  0.160000  0.110000   
LiveJournal   0.480000  0.390000  0.280000  0.210000  0.160000  0.110000   
NetHEP        0.740000  0.520000  0.510000  0.280000  0.300000  0.150000   
NetPhy        0.740000  0.520000  0.500000  0.280000  0.300000  0.150000   
Orkut         0.480000  0.390000  0.280000  0.210000  0.160000  0.110000   
Pokec         0.480000  0.390000  0.280000  0.210000  0.160000  0.110000   
Slashdot0811  0.480000  0.390000  0.280000  0.210000  0.160000  0.110000   
Slashdot0902  0.480000  0.390000  0.280000  0.210000  0.160000  0.110000   
Twitter       0.540000  0.420000  0.340000  0.220000  0.200000  0.110000   
Youtube       0.480000  0.390000  0.280000  0.210000  0.160000  0.110000   
Mean          0.532308  0.416154  0.325385  0.223077  0.188462  0.116923   

Setting         0.01                                                      \
Split            2                   4                   8                 
               Naive     Sorted    Naive     Sorted    Naive     Sorted    
Dataset                                                                    
Amazon        0.810000  0.510000  0.610000  0.280000  0.370000  0.160000   
DBLP          0.730000  0.500000  0.500000  0.270000  0.290000  0.150000   
Epinions      0.740000  0.500000  0.500000  0.270000  0.290000  0.150000   
Friendster    0.730000  0.500000  0.500000  0.270000  0.290000  0.150000   
LiveJournal   0.730000  0.500000  0.500000  0.270000  0.290000  0.150000   
NetHEP        0.940000  0.530000  0.790000  0.300000  0.500000  0.180000   
NetPhy        0.940000  0.520000  0.790000  0.300000  0.500000  0.170000   
Orkut         0.730000  0.500000  0.500000  0.270000  0.290000  0.150000   
Pokec         0.730000  0.500000  0.500000  0.270000  0.290000  0.150000   
Slashdot0811  0.730000  0.500000  0.500000  0.270000  0.290000  0.150000   
Slashdot0902  0.730000  0.500000  0.500000  0.270000  0.290000  0.150000   
Twitter       0.930000  0.590000  0.710000  0.340000  0.460000  0.200000   
Youtube       0.730000  0.500000  0.500000  0.270000  0.290000  0.150000   
Mean          0.784615  0.511538  0.569231  0.280769  0.341538  0.158462   

Setting       0.1                                            Mean    
Split          2               4            8                        
             Naive   Sorted  Naive Sorted Naive   Sorted    Naive    
Dataset                                                              
Amazon         1.0  0.640000   1.0  0.52    1.0  0.380000  0.660000  
DBLP           1.0  0.600000   1.0  0.46    1.0  0.340000  0.604444  
Epinions       1.0  0.600000   1.0  0.46    1.0  0.340000  0.605556  
Friendster     1.0  0.600000   1.0  0.46    1.0  0.340000  0.604444  
LiveJournal    1.0  0.600000   1.0  0.46    1.0  0.340000  0.604444  
NetHEP         1.0  0.690000   1.0  0.64    1.0  0.450000  0.753333  
NetPhy         1.0  0.700000   1.0  0.64    1.0  0.450000  0.752222  
Orkut          1.0  0.600000   1.0  0.46    1.0  0.340000  0.604444  
Pokec          1.0  0.600000   1.0  0.46    1.0  0.340000  0.604444  
Slashdot0811   1.0  0.600000   1.0  0.46    1.0  0.340000  0.604444  
Slashdot0902   1.0  0.600000   1.0  0.46    1.0  0.340000  0.604444  
Twitter        1.0  0.720000   1.0  0.56    1.0  0.400000  0.735556  
Youtube        1.0  0.600000   1.0  0.46    1.0  0.340000  0.604444  
Mean           1.0  0.626923   1.0  0.50    1.0  0.364615  0.641709

In [91]:
latex_str = pivot2.to_latex(float_format=lambda x: '{:,.0%}'.format(x),na_rep='-',multirow=True)
# splits = latex_str.split('\n')
splits = latex_str.replace('{l}','{c}').split('\n')
a = splits[7:-3]
b = []
for i in a:
    cells = i.split('&')
    if cells[0]!= 'Mean':
        cells[0] = '\\tt{'+cells[0]+'}'; 
    cells = "&".join(cells)
    b.append(cells)
splits[7:-3] = b
print('\n'.join(splits))

\begin{tabular}{lrrrrrrrrrrrrrrrrrrr}
\toprule
Setting & \multicolumn{6}{c}{0.005} & \multicolumn{6}{c}{0.01} & \multicolumn{6}{c}{0.1} &  Mean \\
Split & \multicolumn{2}{c}{2} & \multicolumn{2}{c}{4} & \multicolumn{2}{c}{8} & \multicolumn{2}{c}{2} & \multicolumn{2}{c}{4} & \multicolumn{2}{c}{8} & \multicolumn{2}{c}{2} & \multicolumn{2}{c}{4} & \multicolumn{3}{c}{8} \\
{} & Naive & Sorted & Naive & Sorted & Naive & Sorted & Naive & Sorted & Naive & Sorted & Naive & Sorted & Naive & Sorted & Naive & Sorted & Naive & Sorted & Naive \\
Dataset      &       &        &       &        &       &        &       &        &       &        &       &        &       &        &       &        &       &        &       \\
\midrule
\tt{Amazon       }&  58\%  &  44\%   &  36\%  &  23\%   &  21\%  &  12\%   &  81\%  &  51\%   &  61\%  &  28\%   &  37\%  &  16\%   &  100\% &  64\%   &  100\% &  52\%   &  100\% &  38\%   &  66\%  \\
\tt{DBLP         }&  48\%  &  39\%   &  28\%  &  21\%   &  16\%  &  11\%  